In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
maxForce = 3.3e3
bucklingForce = 3e3

In [3]:
#Whiffle tree

attachmentPositions = np.array([[-3e3, 1.0375e3, 1.0375e3, 0.6166e3, 0.30833e3], [0, 0.45, 1.05, 1.75, 2.35]], dtype=np.float32)

In [93]:
#Shear force in beam

def shearDiagram(position):
    shearAtPosition = -sum(attachmentPositions[0][position >= attachmentPositions[1][:]])
    return shearAtPosition
    
#Bending force in the beam

def bendingDiagramTest(position):
    mask = np.array([position[:, np.newaxis] >= attachmentPositions[1][:]])
    mask = np.squeeze(mask, axis=0)
    newPos = attachmentPositions[0]
    print(mask.shape)
    print(newPos, mask)
    #print(newPos[np.ix_(*mask.T)])
    #bendingAtPosition = 3e3 * 1.12 - sum(attachmentPositions)
    #bendingAtPosition = 3e3 * 1.12 - sum(attachmentPositions[0, np.newaxis][mask] * attachmentPositions[1][mask[:,]])
    #return bendingAtPosition
    
def bendingDiagram(position):
    bendingAtPosition = 3e3 * 1.12 - sum(attachmentPositions[0][position >= attachmentPositions[1][:]] * attachmentPositions[1][position >= attachmentPositions[1][:]])
    return bendingAtPosition


In [84]:
?np.ix_

In [94]:
print(bendingDiagramTest(lengthBeam))

(270, 5)
[-3000.    1037.5   1037.5    616.6    308.33] [[ True False False False False]
 [ True False False False False]
 [ True False False False False]
 ...
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]]


ValueError: Cross index must be 1 dimensional

In [6]:
#Second moment of area calculation


#stringerParameters = np.array([[1, 1.25, 1.5, 0.152], np.ones(4)*0.02, np.ones(4)*0.02, np.ones(4)*1.5e-3], dtype=np.float32)

#sheetRParameters = np.array([[0.1484, 0.1484, 0.4, 0.1484], [8e-4, 8e-4, 8e-4, 8e-4]], dtype = np.float32)

#Format for elements xCg, yCg, xLength, yLength

#Bottom part stringer and then vertical part stringer
stringerGeneral = np.array([[0.01075, 0.00075, 0.0185, 0.0015], [0.00075, 0.01, 0.0015, 0.02]], dtype=np.float32)
stringerBot = np.array([[0.01075, 0.00075+0.0008, 0.0185, 0.0015], [0.00075, 0.01+0.0008, 0.0015, 0.02]], dtype=np.float32)
stringerTop = np.array([[0.01075, 0.15-0.00075-0.0008, 0.0185, 0.0015], [0.00075, 0.15-0.01-0.0008, 0.0015, 0.02]], dtype=np.float32)

#Sheet order bottom C, left A, top C, right A
sheet = np.array([[0.2, 0.0004, 0.4, 0.0008], [0.0004, 0.075, 0.0008, 0.1484], [0.2, 0.1496, 0.4, 0.0008], [0.3996, 0.075, 0.0008, 0.1484]], dtype=np.float32)

def cgCalculator(structuralElements):
    structuralElements = np.concatenate((structuralElements, np.array([structuralElements[:, 2]*structuralElements[:, 3]]).T), axis=1)
    xCg = (sum(structuralElements[:, 0]*structuralElements[:, 4])/sum(structuralElements[:, 4]))
    yCg = (sum(structuralElements[:, 1]*structuralElements[:, 4])/sum(structuralElements[:, 4]))
    return xCg, yCg

def secondMomentArea(xLength, yLength, xParallelAxis, yParallelAxis, axis):
    if(axis == 'x'):
        return ((yLength**3*xLength)/12 + xLength*yLength*yParallelAxis**2)
    else:
        return ((yLength*xLength**3)/12 + xLength*yLength*xParallelAxis**2) 

def secondMomentAreaAssembly(structuralElements, axisSMA): #structuralElements[xCg, yCg, xLength, yLength, ]
    cg = cgCalculator(structuralElements)

    return sum(secondMomentArea(structuralElements[:, 2], structuralElements[:, 3], abs(cg[0]-structuralElements[:, 0]), abs(cg[1]-structuralElements[:, 1]), axisSMA))

def cutElements(structuralElements, axis, height):
    if(axis == 'x'):
        lowerCorner = structuralElements[:, 0] -  structuralElements[:, 2]/2
        heightMask = (lowerCorner) < height
        truncatedStructuralElements = structuralElements[heightMask].copy()
        truncatedStructuralElements[:, 2] = np.minimum(truncatedStructuralElements[:, 2], (height - lowerCorner[heightMask]))
        truncatedStructuralElements[:, 0] = (lowerCorner[heightMask] + truncatedStructuralElements[:, 2]/2)
        return truncatedStructuralElements
    else:
        lowerCorner = structuralElements[:, 1] -  structuralElements[:, 3]/2
        heightMask = (lowerCorner) < height
        truncatedStructuralElements = structuralElements[heightMask].copy()
        truncatedStructuralElements[:, 3] = np.minimum(truncatedStructuralElements[:, 3], (height - lowerCorner[heightMask]))
        truncatedStructuralElements[:, 1] = (lowerCorner[heightMask] + truncatedStructuralElements[:, 3]/2)
        return truncatedStructuralElements
    
def firstMomentAreaAssembly(structuralElements, axis, height):
    cgReference = cgCalculator(structuralElements)
    if(axis == 'x'):
        structuralElementsCut = cutElements(structuralElements, 'y', height)
        structuralElementsCut = np.concatenate((structuralElementsCut, np.array([structuralElementsCut[:, 2]*structuralElementsCut[:, 3]]).T), axis=1)
        cgPart = cgCalculator(structuralElementsCut)[1]
        return abs(cgReference[1]-cgPart)*sum(structuralElementsCut[:, 2]*structuralElementsCut[:, 3])
    else:
        structuralElementsCut = cutElements(structuralElements, 'x', height)
        structuralElementsCut = np.concatenate((structuralElementsCut, np.array([structuralElementsCut[:, 2]*structuralElementsCut[:, 3]]).T), axis=1)
        cgPart = cgCalculator(structuralElementsCut)[0]
        return abs(cgReference[0]-cgPart)*sum(structuralElementsCut[:, 2]*structuralElementsCut[:, 3])

In [17]:
#print(secondMomentAreaAssembly(stringer, 'x'))

nbStringersTop = 3
nbStringersBot = 2

stringerAssemblyTop = np.repeat(stringerTop, [nbStringersTop, nbStringersTop], axis=0)
stringerAssemblyBot = np.repeat(stringerBot, [nbStringersBot, nbStringersBot], axis=0)

wingAssembly = np.concatenate((sheet, stringerAssemblyBot, stringerAssemblyTop), axis=0)

print(secondMomentAreaAssembly(stringerGeneral, 'x'))
print(secondMomentAreaAssembly(sheet, 'x'))
print(secondMomentAreaAssembly(wingAssembly, 'x'))
print(firstMomentAreaAssembly(wingAssembly, 'x', 0.075))


2.238636276352679e-09
3.997489443463564e-06
5.355828626818493e-06
3.8088949885803754e-05


In [10]:
#Max bending moment calculation

def normalBendingStress(positionFromRoot, structuralElements, axis):
    if(axis == 'x'):
        SMA = secondMomentAreaAssembly(structuralElements, axis)
        neutralAxisPosition = cgCalculator(structuralElements)[1]
        if(neutralAxisPosition > 0.15 - neutralAxisPosition):
            return bendingDiagram(positionFromRoot)*neutralAxisPosition/SMA
        else:
            return bendingDiagram(positionFromRoot)*(0.15-neutralAxisPosition)/SMA
    else:  #this part is not yet finished
        SMA = secondMomentAreaAssembly(structuralElements, axis)
        neutralAxisPosition = cgCalculator(structuralElements)[0]
        if(neutralAxisPosition > 0.15 - neutralAxisPosition):
            return bendingDiagram(positionFromRoot)*neutralAxisPosition/SMA
        else:
            return bendingDiagram(positionFromRoot)*(0.15-neutralAxisPosition)/SMA
        
def shearStress(positionFromRoot, structuralElements, height, axis):
    SMA = secondMomentAreaAssembly(structuralElements, axis)
    FMA = firstMomentAreaAssembly(structuralElements, axis, height)
    print(SMA, FMA)
    maxShear = shearDiagram(positionFromRoot)*FMA/(SMA*0.0015)
    
    return maxShear

In [18]:
print(f'Normal stress: {normalBendingStress(0.001, wingAssembly, "x")/1e6} MPa')

print(f'Shear stress: {shearStress(1.3, wingAssembly, 0.075, "x")/1e6} MPa')

Normal stress: 49.18411212812735 MPa
5.355828626818493e-06 3.8088949885803754e-05
Shear stress: 4.385537215529791 MPa


In [21]:
normalBendingStress(0.001, wingAssembly, "x")

49460826.61770038

In [20]:
#Vizualisation 

lengthBeam = np.arange(0.001, 2.7, 0.01)
heightBeam = np.arange(0.001, 0.15, 0.01)

plt.plot(lengthBeam, normalBendingStress(lengthBeam, wingAssembly, "x")/1e6)
# plt.show()
print(lengthBeam)

ValueError: operands could not be broadcast together with shapes (270,) (5,) 